In [5]:
import os
import re
import pandas as pd
import numpy as np
import openpyxl

In [13]:
def load_data(folder_path, pattern_prefix):
    """
    Load data from Excel files with flexible naming pattern
    
    Args:
        folder_path: Path to directory containing Excel files
        pattern_prefix: The middle part of the filename (e.g., "Location" or "Scenarios")
                       Files should be named like f"SA_{pattern_prefix}_*.xlsx"
    
    Returns:
        Nested dictionary: {name: {sheet: DataFrame}}
    """
    # Verify the folder exists
    if not os.path.exists(folder_path):
        raise FileNotFoundError(f"The directory {folder_path} does not exist")
    
    # Create regex pattern to extract names
    pattern = re.compile(fr'SA_{pattern_prefix}_(.*?)\.xlsx')
    
    # Get all names from filenames
    names = []
    for f in os.listdir(folder_path):
        match = pattern.match(f)
        if match:
            names.append(match.group(1))
    
    # Initialize nested dictionary
    data_dict = {name: {} for name in names}
    
    # Read each file and its sheets
    for name in data_dict.keys():
        filename = f"SA_{pattern_prefix}_{name}.xlsx"
        file_path = os.path.join(folder_path, filename)
        
        try:
            # Read all sheets from the Excel file
            sheets = pd.read_excel(file_path, sheet_name=None)
            
            # Store each sheet (Fold_1 to Fold_5) in the dictionary
            for sheet_num in range(1, 6):
                sheet_name = f"Fold_{sheet_num}"
                if sheet_name in sheets:
                    data_dict[name][sheet_name] = sheets[sheet_name]
                else:
                    print(f"Warning: Sheet {sheet_name} not found in {filename}")
                    
        except FileNotFoundError:
            print(f"Warning: File {filename} not found in {folder_path}")
        except Exception as e:
            print(f"Error processing {filename}: {str(e)}")
    
    return data_dict

def average_folds(data_dict):
    """
    Calculate the average of all folds for each location.
    
    Args:
        data_dict (dict): Dictionary with location as keys and fold data as values
        
    Returns:
        dict: Dictionary with location as keys and averaged DataFrame as values
    """
    result = {}
    
    for location, fold_data in data_dict.items():
        # Collect all dataframes from folds
        all_folds = []
        
        for fold_name, df in fold_data.items():
            if isinstance(df, pd.DataFrame):
                all_folds.append(df)
        
        if not all_folds:
            continue
        
        # Concatenate all folds
        combined_df = pd.concat(all_folds, ignore_index=True)
        
        # Group by algorithm and calculate mean for numeric columns
        numeric_cols = combined_df.select_dtypes(include=[np.number]).columns
        avg_df = combined_df.groupby('Algorithm')[numeric_cols].mean().reset_index()
        
        result[location] = avg_df
    
    return result


def create_testing_cost_dataframe(averaged_results):
    """
    Create a pandas DataFrame with algorithms as index, locations as columns, and Testing Total Cost.
    
    Args:
        averaged_results (dict): Dictionary with location as keys and averaged DataFrames as values
        
    Returns:
        pd.DataFrame: Formatted table with testing costs
    """
    # Initialize data structure
    data_dict = {}
    
    for location, df in averaged_results.items():
        for _, row in df.iterrows():
            algorithm = row['Algorithm']
            testing_cost = row['Testing Total Cost']
            
            if algorithm not in data_dict:
                data_dict[algorithm] = {}
            
            data_dict[algorithm][location] = testing_cost
    
    # Convert to DataFrame
    result_df = pd.DataFrame.from_dict(data_dict, orient='index')
    
    # Sort rows (algorithms) and columns (locations)
    result_df = result_df.sort_index()  # Sort algorithms alphabetically
    result_df = result_df.reindex(sorted(result_df.columns), axis=1)  # Sort locations alphabetically
    
    # Format the values for better readability
    formatted_df = result_df.copy()
    for col in formatted_df.columns:
        formatted_df[col] = formatted_df[col].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")
    
    return result_df, formatted_df

In [14]:
SA_name = 'Locations'

# Example usage for Locations:
location_data = load_data(
    folder_path=os.path.join('Safe_Results', f'SA_{SA_name}'),
    pattern_prefix=SA_name
)

# Example usage:
averaged_results_loc = average_folds(location_data)

# Example usage:
numeric_df_loc, formatted_df_loc = create_testing_cost_dataframe(averaged_results_loc)
print("Numeric DataFrame (for calculations):")
print(numeric_df_loc)
print("\nFormatted DataFrame (for display):")
print(formatted_df_loc)


Numeric DataFrame (for calculations):
         Alaska     Arizona  California     Florida   Minnesota
LP  17485.02388  2726.54760  2493.26588  2240.03700  6446.31336
RO  18038.33124  2743.89756  2429.77244  2206.52824  6550.79844
SO  17500.63656  2726.50428  2401.52664  2183.81084  6446.38356

Formatted DataFrame (for display):
        Alaska    Arizona California    Florida  Minnesota
LP  $17,485.02  $2,726.55  $2,493.27  $2,240.04  $6,446.31
RO  $18,038.33  $2,743.90  $2,429.77  $2,206.53  $6,550.80
SO  $17,500.64  $2,726.50  $2,401.53  $2,183.81  $6,446.38


In [16]:
SA_name = 'Scenarios'

# Example usage for Locations:
scenario_data = load_data(
    folder_path=os.path.join('Safe_Results', f'SA_{SA_name}'),
    pattern_prefix=SA_name
)

# Example usage:
averaged_results_sce = average_folds(scenario_data)

# Example usage:
numeric_df_sce, formatted_df_sce = create_testing_cost_dataframe(averaged_results_sce)
print("Numeric DataFrame (for calculations):")
print(numeric_df_sce)
print("\nFormatted DataFrame (for display):")
print(formatted_df_sce)

Numeric DataFrame (for calculations):
              DC         FOB          RC
LP  699623.35560  2493.26588  3830.74292
RO  783407.88288  2429.77244  3515.65216
SO  696714.48520  2401.52664  3418.08168

Formatted DataFrame (for display):
             DC        FOB         RC
LP  $699,623.36  $2,493.27  $3,830.74
RO  $783,407.88  $2,429.77  $3,515.65
SO  $696,714.49  $2,401.53  $3,418.08


In [17]:
SA_name = 'Capacity_Costs'

# Example usage for Locations:
cap_cost_data = load_data(
    folder_path=os.path.join('Safe_Results', f'SA_{SA_name}'),
    pattern_prefix=SA_name
)

# Example usage:
averaged_results_cap_cost = average_folds(cap_cost_data)

# Example usage:
numeric_df_cap_cost, formatted_df_cap_cost = create_testing_cost_dataframe(averaged_results_cap_cost)
print("Numeric DataFrame (for calculations):")
print(numeric_df_cap_cost)
print("\nFormatted DataFrame (for display):")
print(formatted_df_cap_cost)



Numeric DataFrame (for calculations):
          HPHB        HPLB        LPHB        LPLB
LP  2493.26588  2087.28504  1949.75136  1661.06536
RO  2429.77244  2109.38756  1905.26892  1719.53312
SO  2401.52664  2087.33548  1870.37588  1660.04368

Formatted DataFrame (for display):
         HPHB       HPLB       LPHB       LPLB
LP  $2,493.27  $2,087.29  $1,949.75  $1,661.07
RO  $2,429.77  $2,109.39  $1,905.27  $1,719.53
SO  $2,401.53  $2,087.34  $1,870.38  $1,660.04
